In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import collections
from datetime import datetime
from IPython.display import clear_output, display, HTML
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
from pathlib import Path
import pickle
import scipy
# import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
import sys
from numpy.fft import fft, rfft
from scipy.signal import spectrogram
from wfOpto import *
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings("ignore")

LED Oscilations Test w/ AL_0030

In [5]:
# load stimulus related files generated post-experiment
pdFlips = np.load(r'Z:\Subjects\AL_0030\2023-01-19\3\ledFlipTimes.npy').ravel()
flipRates = np.load(r'Z:\Subjects\AL_0030\2023-01-19\3\ledFreq.npy').ravel()

def findStimStartTimes():
    """
    This function takes in the photodiode flip times, which includes ALL
    times the light turned on and off (so not just the start of the stimulus).
    Taking advantage of the fact that stimuli are exactly 2 seconds long, we
    find the flip times that do correspond to the start of each stimulus.
    
    No input arguments - uses loaded files above.
    Returns:
        startTimes: array of the start time for each stimulus. e.g. if 
            there were 10 repetitions each of 5 different frequencies,
            this would be length (50, 1)
        stimFlips: array of the flip times within each stimulus. e.g.
            in the above example, it would be length (50, n), where
            n corresponds to the number of flips that occurred within
            the stimulus (and thus is not even across stimuli)
    """
    
    durationS = 2 # fixed?
    startTimes = []
    stimFlips = []
    
    thisStart = pdFlips[0]

    for stim in range(len(flipRates)):
        thisEnd = thisStart + durationS + 0.1
        try:
            nextStart = pdFlips[pdFlips > thisEnd][0] # set to first pd flip that is past the stim duration
        except:
            nextStart = None # should only happen at the end?
        startTimes.append(thisStart)
        
        flipIx = np.where((pdFlips >= thisStart) & (pdFlips <= thisEnd))
        flipTimes = pdFlips[flipIx]
        stimFlips.append(flipTimes)
        
        thisStart = nextStart
        
    startTimes = np.array(startTimes)
    stimFlips = np.array(stimFlips, dtype='object')
    return startTimes, stimFlips


startTimes, stimFlips = findStimStartTimes()

In [6]:
serverPath = Path(r'Z:\Subjects\AL_0030\2024-01-19\3')
timeFile = serverPath / 'cameraFrameTimes.npy'
frameTimes = np.squeeze(np.load(timeFile))[::2] # every other frame - we want blue only
svdTemp = np.load(serverPath / 'corr/svdTemporalComponents_corr.npy')
svdSpat = np.load(serverPath / 'blue/svdSpatialComponents.npy')
svdSpatFull = svdSpat[:,:,:500]
meanImage = np.load(serverPath / 'blue/meanImage.npy')
px, py, ncomps = svdSpatFull.shape
svdSpat = svdSpatFull.reshape(px*py, ncomps)
tToWf = scipy.interpolate.interp1d(frameTimes, svdTemp, axis=0, fill_value='extrapolate')
spatial = svdSpatFull.reshape(560*560,-1)

ValueError: x and y arrays must be equal in length along interpolation axis.

In [ ]:
#collecting 

allVideos=[]
freqActivityLeft=[]
freqActivityRight=[]
brain=[]
for count,freq in enumerate(np.unique(flipRates)[3:7]):
    index = [i for i,x in enumerate(flipRates) if x == freq]
    startTimesFreq = startTimes[index]
    allVideos = []
    time =  [np.linspace(i+0, i+2, 70) for i in startTimesFreq]
    activity = tToWf(time)
    activity = np.mean(activity, axis=0)
    dwf = [np.diff(i, prepend=i[0]) for i in activity.T]
    dwf = np.array(dwf)

    video = spatial @ dwf
    video = video.reshape(560,560,-1)
    
    # brain.append(np.mean(video,axis=2)) # brain image for all 30 reps, all 100 time points - average activity during a rep
    
    vcLeft = np.mean(video[380:500,120:220,:],axis=(0,1)) # left visual cortex
    freqActivityLeft.append(vcLeft)
    vcRight = np.mean(video[380:500,350:470,:],axis=(0,1)) # right visual cortex
    freqActivityRight.append(vcRight)
    
freqActivityLeft=np.array(freqActivityLeft)
freqActivityRight=np.array(freqActivityRight)
# brain=np.array(brain)

In [ ]:
# fouriers and plotting
fig, axs = plt.subplots(3,2,figsize=(7,7))
t = np.linspace(0,2,70) #i think we define our trial times and thus our sampling rate?
dt = t[1]-t[0] 

for i,freq in enumerate(np.unique(flipRates)[3:6]):
    freq =np.unique(flipRates)[i+3]
    x = freqActivityLeft[i]

    x = x.reshape(-1)
    N=x.shape[0]
    T = N*dt
    df = 1/T.max() # our resolution
    
    xf = fft(x - x.mean()) #fourier transform
    Sxx = 2 * dt * 2/T * (xf *  xf.conj()) #spectrum
    Sxx = Sxx[:int(len(x)/2)] # our data is real, so ignore negative freqs
    fNQ = 1/dt/2 # nyquist freq - expected highest freq
    faxis = np.arange(0,fNQ,df)

    xmax = x.max()
    max=Sxx[np.where(Sxx==xmax)]

    max_y = Sxx.reshape(-1).max()  # Find the maximum y value
    max_x = faxis[Sxx.argmax()]

    axs[i,0].plot(faxis,Sxx)
    axs[i,0].set_xlabel('frequency')
    axs[i,0].set_ylabel('Power [$\mu V^2$/Hz]')
    axs[i,0].set_title(f'freq {freq} left side')
    axs[i,0].autoscale(axis='x',tight='true')
    axs[i,0].set_ylim([0,5000])
    axs[i,0].text(s=f'max: {max_x:.2f}',x=10,y=10)

    x = freqActivityRight[i]
    x = x.reshape(-1)
    xmax = x.max()
    xf = fft(x - x.mean()) 
    Sxx = 2 * dt * 2/T * (xf *  xf.conj()) 
    Sxx = Sxx[:int(len(x)/2)]
    fNQ = 1/dt/2
    faxis = np.arange(0,fNQ,df)

    max_y = Sxx.reshape(-1).max()  # Find the maximum y value
    max_x = faxis[Sxx.argmax()]

    axs[i,1].plot(faxis,Sxx)
    axs[i,1].set_xlabel('frequency')
    axs[i,1].set_title(f'freq {freq} right side')
    axs[i,1].autoscale(axis='x',tight='true')
    axs[i,1].set_ylim([0,5000])
    axs[i,1].text(s=f'max: {max_x:.2f}',x=10,y=10)

    fig.tight_layout()